In [10]:

import pandas as pd

from glob import glob
import os 
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification


In [11]:
r = glob('chatting_datas/labeled_datas/*.csv')
data = pd.DataFrame()
for i in r:
    d = pd.read_csv(i)
    data = pd.concat([data, d], axis = 0)
data = data.reset_index(drop = True)
labels = data['label']
message = list(data['message'])


In [12]:
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
    
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_texts = tokenize_function(message)

In [13]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_texts), labels))
dataset = dataset.shuffle(len(message)).batch(32)


In [14]:

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(dataset, epochs = 2)


Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 24/221 [==>...........................] - ETA: 33:55 - loss: 0.6305 - accuracy: 0.6419

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(dataset)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# 새로운 텍스트 예측
new_texts = ["Example text to classify"]
new_tokenized_texts = tokenize_function(new_texts)
predictions = model.predict(dict(new_tokenized_texts))
predicted_labels = tf.argmax(predictions.logits, axis=-1).numpy()
print(predicted_labels)
